In [1]:
# Jupyter Notebook settings
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import base64
import os
import shutil
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import src
import sys
import time
from datetime import datetime
import pytz
from io import BytesIO
from PIL import Image

from keras.applications.imagenet_utils import preprocess_input
from keras.backend.tensorflow_backend import set_session
from keras.preprocessing import image
import numpy as np
import tensorflow as tf

from ssd import SSD300
from ssd_utils import BBoxUtility

def save_image(src, file_save_path):
    # Base64エンコードされた画像をデコードして保存する。
    if "base64," in src:
        with open(file_save_path, "wb") as f:
            f.write(base64.b64decode(src.split(",")[1]))

    # 画像のURLから画像を保存する。
    else:
        res = requests.get(src, stream=True)
        with open(file_save_path, "wb") as f:
            shutil.copyfileobj(res.raw, f)

Using TensorFlow backend.
C:\Users\kimura\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kimura\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kimura\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kimura\Anaconda3\lib\site-packages\tensorflow\python\framework\dtype

In [3]:
# Main
# logger setting
logger = src.Setup_Logger.Setup_Logger()
np.set_printoptions(suppress=True)
# SSD setting
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.45
set_session(tf.Session(config=config))
voc_classes = ['Aeroplane', 'Bicycle', 'Bird', 'Boat', 'Bottle',
               'Bus', 'Car', 'Cat', 'Chair', 'Cow', 'Diningtable',
               'Dog', 'Horse', 'Motorbike', 'Person', 'Pottedplant',
               'Sheep', 'Sofa', 'Train', 'Tvmonitor']
NUM_CLASSES = len(voc_classes) + 1
input_shape = (300, 300, 3)
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('weights/weights_SSD300.hdf5', by_name=True)
bbox_util = BBoxUtility(NUM_CLASSES)

W0109 11:13:39.965100 12096 deprecation_wrapper.py:119] From C:\Users\kimura\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0109 11:13:39.967098 12096 deprecation_wrapper.py:119] From C:\Users\kimura\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0109 11:13:39.969094 12096 deprecation_wrapper.py:119] From C:\Users\kimura\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0109 11:13:39.994026 12096 deprecation_wrapper.py:119] From C:\Users\kimura\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0109 11:13:40.446613 12096 deprecation_wrapper.py:119] From C:\Users\kimur

In [4]:
# Chronium setting
user,pw,ip = src.Setup_Config.Setup_Config(logger=logger)
main_url = 'http://' + user + ':' + pw + '@' + ip + ':15790'
options = Options()
options.binary_location = None
if os.name == 'posix':
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
elif os.name == 'nt':
    options.binary_location = 'C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe'

In [5]:
if options.binary_location is None:
    logger.info('Support : Windows , OSX')
    sys.exit()
try:
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    driver.get(main_url)
    logger.info('Accessing')

    start_button = driver.find_element_by_id('VMG_PRE_START_BTN')
    start_button.click()
    
    print("Start Detector")
    while True:        
        try:
            BUF_UVCCAM2 = driver.find_element_by_id('BUF_UVCCAM2')
            img_url = BUF_UVCCAM2.get_attribute('src')        

            img_response = requests.get(img_url)
            output_img = Image.open(BytesIO(img_response.content))
            
            img = output_img.copy()
            img = img.resize((300,300))
            img = image.img_to_array(img)
            
            inputs = []
            inputs.append(img.copy())
        except Exception as e:
            continue
        
        
        inputs = preprocess_input(np.array(inputs))
        preds = model.predict(inputs, batch_size=1)
        results = bbox_util.detection_out(preds)

        # Parse the outputs.
        det_label = results[0][:, 0]
        det_conf = results[0][:, 1]

        cand_size = len(det_label)
        isPerson = False
        
        for i in range(cand_size):
            if det_conf[i] < 0.9:
                continue
            else:
                # Person is 15
                if int(det_label[i]) == 15:
                    isPerson = True
                    break
                    
        if isPerson:
            # time de hozon 
            now_time_str = datetime.now(pytz.timezone('Asia/Tokyo')).strftime('%Y%m%d_%H%M%S%f')[:-3]
            # print("Person detect :",now_time_str)
            
            try:
                image.save_img("./output/" +now_time_str + '.jpg', output_img)
            except Exception as e:
                continue
        time.sleep(0.5)
        
except Exception as e:
    logger.info('Web driver Error occured!')
    logger.exception(e)
    sys.exit()

2020-01-09 11:13:57,758:66:INFO:Error occured!
I0109 11:13:57.758676 12096 <ipython-input-5-3335ba21dfda>:66] Error occured!
2020-01-09 11:13:57,758:67:ERROR:Message: session not created: This version of ChromeDriver only supports Chrome version 74
  (Driver info: chromedriver=74.0.3729.6 (255758eccf3d244491b8a1317aa76e1ce10d57e9-refs/branch-heads/3729@{#29}),platform=Windows NT 10.0.18362 x86_64)
Traceback (most recent call last):
  File "<ipython-input-5-3335ba21dfda>", line 6, in <module>
    driver = webdriver.Chrome(options=options)
  File "C:\Users\kimura\Anaconda3\lib\site-packages\selenium\webdriver\chrome\webdriver.py", line 81, in __init__
    desired_capabilities=desired_capabilities)
  File "C:\Users\kimura\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 157, in __init__
    self.start_session(capabilities, browser_profile)
  File "C:\Users\kimura\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 252, in start_session
    re

SystemExit: 

C:\Users\kimura\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
